<a href="https://colab.research.google.com/github/Spratumn/CameraCalibrate/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

加载Google云盘

In [0]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
import os  # file path
print(os.listdir(os.curdir))

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
['.config', 'drive', 'sample_data']


import pakages

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader
from imgaug import augmenters as iaa
import cv2 as cv
import random
import shutil
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt

配置信息

In [0]:
class Config():
    # model config
    CLASS_NUM = 8
    
    # deeplabv3p
    ASPP_OUT_CHANNEL = 256
    SHORTCUT_CHANNEL = 48
    
    # train config
    MODEL = 'deeplabv3+' # 'unet','deeplabv3+'
    TRAIN_NUMBER = 5 # 1,2,3,4,5,6,7,8,9,10
    EPOCHS = 2  # total->50
    TRAIN_BATCH_SIZE = 8
    EVAL_BATCH_SIZE = 4
    WEIGHT_DECAY = 1.0e-4
    LR_LIST = [3e-4,2e-4,1e-4,5e-5,2e-5,1e-5,5e-6,2e-6,1e-6,5e-7]
    BASE_LR = LR_LIST[TRAIN_NUMBER]
    CSV_DIR = "/content/drive/My Drive/LaneSeg/data_list"
    LOG_DIR = "/content/drive/My Drive/LaneSeg/logs"
    PRE_TRAINED = True
    PRE_TRAIN_WEIGHTS = 'laneNet_deeplabv3+_5th_epoch_02.pth.tar' # 'laneNet_{0}_{1}th.pth.tar'.format(MODEL,TRAIN_NUMBER-1)

    # dataset config
    IMAGE_SHAPE = (3,1710,3384)
    CROP_SIZE = 699
    RESIZE_SCALE = 3
    # train_img_size->(3,337,1128)

cfg = Config()

utils—label_process

In [0]:
def label_encoder(mark_label):
    """mark label to train label"""

    train_label = np.zeros((mark_label.shape[0], mark_label.shape[1]))

    id_train = {0:[0, 249, 255, 213, 206, 207, 211, 208,216,215,218, 219,232, 202, 231,230,228,229,233,212,223],
                1:[200, 204, 209], 2: [201,203], 3:[217], 4:[210], 5:[214],
                6:[220,221,222,224,225,226], 7:[205,227,250]}
    for i in range(8):
        for item in id_train[i]:
            train_label[mark_label == item] = i

    return train_label


def label_id_decoder(train_label):
    """train label to mark label for id"""
    mark_label = np.zeros((train_label.shape[0], train_label.shape[1]), dtype='uint8')
    # 0
    mark_label[train_label == 0] = 0
    # 1
    mark_label[train_label == 1] = 204
    # 2 
    mark_label[train_label == 2] = 203
    # 3
    mark_label[train_label == 3] = 217
    # 4
    mark_label[train_label == 4] = 210
    # 5
    mark_label[train_label == 5] = 214
    # 6
    mark_label[train_label == 6] = 224
    # 7
    mark_label[train_label == 7] = 227

    return mark_label


def label_color_decoder(train_label):
    """train label to mark label for color map"""
    mark_label = np.zeros((3, train_label.shape[0], train_label.shape[1]), dtype='uint8')
    # 0
    mark_label[0][train_label == 0] = 0
    mark_label[1][train_label == 0] = 0
    mark_label[2][train_label == 0] = 0
    # 1
    mark_label[0][train_label == 1] = 70
    mark_label[1][train_label == 1] = 130
    mark_label[2][train_label == 1] = 180
    # 2
    mark_label[0][train_label == 2] = 0
    mark_label[1][train_label == 2] = 0
    mark_label[2][train_label == 2] = 142
    # 3
    mark_label[0][train_label == 3] = 153
    mark_label[1][train_label == 3] = 153
    mark_label[2][train_label == 3] = 153
    # 4
    mark_label[0][train_label == 4] = 128
    mark_label[1][train_label == 4] = 64
    mark_label[2][train_label == 4] = 128
    # 5
    mark_label[0][train_label == 5] = 190
    mark_label[1][train_label == 5] = 153
    mark_label[2][train_label == 5] = 153
    # 6
    mark_label[0][train_label == 6] = 0
    mark_label[1][train_label == 6] = 0
    mark_label[2][train_label == 6] = 230
    # 7
    mark_label[0][train_label == 7] = 255
    mark_label[1][train_label == 7] = 128
    mark_label[2][train_label == 7] = 0

    return mark_label
  


utils—image_process

In [0]:
# attention: .jpg read by opencv,  .png read by PIL

# crop the image to discard useless parts
def crop_resize_data(image, label=None):
    """
    Attention:
    h,w, c = image.shape
    cv2.resize(image,(w,h))
    """
    roi_image = image[cfg.CROP_SIZE:, :]  # crop size
    _,h,w = cfg.IMAGE_SHAPE
    image_size = (int((h+1-cfg.CROP_SIZE)/cfg.RESIZE_SCALE),int(w/cfg.RESIZE_SCALE))
    if label is not None:
        roi_label = label[cfg.CROP_SIZE:, :]
        train_image = cv.resize(roi_image, image_size, interpolation=cv.INTER_LINEAR)
        train_label = cv.resize(roi_label, image_size, interpolation=cv.INTER_NEAREST)
        return train_image, train_label
    else:
        train_image = cv.resize(roi_image, image_size, interpolation=cv.INTER_LINEAR)
        return train_image


class LaneSegTrainDataset(Dataset):

    def __init__(self, csv_file, transform=None):

        super(LaneSegTrainDataset, self).__init__()
        # read csv as data index
        self.data_paths = pd.read_csv(os.path.join(cfg.CSV_DIR, csv_file), header=None,
                                names=["image","label"])
        self.image_paths = self.data_paths["image"].values[1:]
        self.label_paths = self.data_paths["label"].values[1:]

        self.transform = transform

    def __len__(self):
        return self.label_paths.shape[0]

    def __getitem__(self, idx):
        # read image and label by index
        ori_image = cv.imread(self.image_paths[idx])
        ori_label = np.array(Image.open(self.label_paths[idx]))
        # crop and resize image and label
        train_img, ori_label = crop_resize_data(ori_image, ori_label)
        # Encode
        train_label = label_encoder(ori_label)

        sample = {'image': train_img,
                'label': train_label}
        if self.transform:
            sample = self.transform(sample)
        
        return sample

class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, label = sample['image'], sample['label']

        # swap color axis because
        # opencv image: H x W x C
        # torch image: C X W X H
        image = image.transpose((2, 0, 1))

        image_tensor = torch.from_numpy(image.astype(np.float32))
        label_tensor = torch.from_numpy(label.astype(np.long))
        return {'image': image_tensor,
                'label': label_tensor}


# imgaug Augmentation

class ImageAug(object):
    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        if np.random.uniform(0,1) > 0.5:
            seq = iaa.Sequential([iaa.OneOf([
                iaa.AdditiveGaussianNoise(scale=(0, 0.2 * 255)),
                iaa.Sharpen(alpha=(0.1, 0.3), lightness=(0.7, 1.3)),
                iaa.GaussianBlur(sigma=(0, 1.0))])])
            image = seq.augment_image(image)
        return {'image': image,
                'label': label}

utils—loss and metric

In [0]:
class MySoftmaxCrossEntropyLoss(nn.Module):

    def __init__(self, nbclasses):
        super(MySoftmaxCrossEntropyLoss, self).__init__()
        self.nbclasses = nbclasses

    def forward(self, inputs, target):
        if inputs.dim() > 2:
            inputs = inputs.view(inputs.size(0), inputs.size(1), -1)  # N,C,H,W => N,C,H*W
            inputs = inputs.transpose(1, 2)  # N,C,H*W => N,H*W,C
            inputs = inputs.contiguous().view(-1, self.nbclasses)  # N,H*W,C => N*H*W,C
        target = target.view(-1)
        return nn.CrossEntropyLoss(reduction="mean")(inputs, target)


def update_confusion_matrix(prediction, label, confus_matrix):
    """
    prediction : [N, H, W]
    label: [N, H, W]
    """
    prediction = prediction.cpu().numpy()
    label = label.cpu().numpy()
    for i in range(8):
        # label: Positive or Negative
        label_p = label == i
        label_n = label != i

        # prediction: Positive or Negative
        pred_p = prediction == i
        pred_n = prediction != i

        # label == Positive and prediction == Positive
        tp = np.sum(label_p * pred_p)
        # label == Negative and prediction == Negative 
        tn = np.sum(label_n * pred_n)
        # label == Negative and prediction == Positive 
        fp = np.sum(label_n * pred_p)
        # label == Positive and prediction == Negative 
        fn = np.sum(label_p * pred_n)
        
        confus_matrix["TP"][i] += tp
        confus_matrix["TN"][i] += tn
        confus_matrix["FP"][i] += fp
        confus_matrix["FN"][i] += fn

    return confus_matrix

def compute_iou(confusion_matrix):
    class_num = len(confusion_matrix["TP"].keys())
    ious = [0.0]*class_num

    for i in range(class_num):
        tp = confusion_matrix["TP"][i]
        fp = confusion_matrix["FP"][i]
        fn = confusion_matrix["FN"][i]
        ious[i] = tp/(tp+fp+fn)
    return ious

def compute_precision(confusion_matrix):
    class_num = len(confusion_matrix["TP"].keys())
    precisions = [0.0]*class_num

    for i in range(class_num):
        tp = confusion_matrix["TP"][i]
        fp = confusion_matrix["FP"][i]
        precisions[i] = tp/(tp+fp)
    return precisions

def compute_recall(confusion_matrix):
    class_num = len(confusion_matrix["TP"].keys())
    recalls = [0.0]*class_num

    for i in range(class_num):
        tp = confusion_matrix["TP"][i]
        fn = confusion_matrix["FN"][i]
        recalls[i] = tp/(tp+fn)
    return recalls

def compute_F1_score(precisions,recalls):
    class_num = len(precisions)
    f1_scores = [0.0]*class_num

    for i in range(class_num):
        f1_scores[i] = 2*precisions[i]*recalls[i]/(precisions[i]+recalls[i])
    return f1_scores

def compute_mean(score_list):
    return np.mean(score_list[1:])

model

In [0]:
"""
SeparableConv2d(in_channel, out_channel)
AtrousConv(in_channel, out_channel, dilation)

"""

class SeparableConv2d(nn.Sequential):
    """
    define separable Convolution Sequential
    """
    def __init__(self,in_channel,out_channel
                ,kernel_size=3,stride=1
                ,padding=0,dilation=1, bias=False,act=0):
        layers = []
        layers.append(nn.Conv2d(in_channel,in_channel
                            ,kernel_size=kernel_size
                            ,stride=stride
                            ,padding=padding
                            ,dilation=dilation
                            ,groups=in_channel
                            ,bias=False))
        layers.append(nn.BatchNorm2d(in_channel))
        if act==1:
            layers.append(nn.ReLU())
        layers.append(nn.Conv2d(in_channel,out_channel
                            ,kernel_size=1
                            ,bias=False))
        if act==1:
            layers.append(nn.ReLU())
        super(SeparableConv2d, self).__init__(*layers)


class AtrousConv(nn.Sequential):
    """
    define atrous Convolution Sequential with specific atrous_rate
    kernel_size = 3  -->  padding = dilation
    stride = 1
    channels no change: out_channel = in_channel
    out_size no change
    """
    def __init__(self, in_channel, out_channel, dilation):
        layers = [
            SeparableConv2d(in_channel, out_channel, kernel_size=3
                    , padding=dilation, dilation=dilation, bias=False,act=0),
            nn.BatchNorm2d(out_channel),
            nn.ReLU()
        ]
        super(AtrousConv, self).__init__(*layers)

"""
ASPPConv(in_channel, out_channel, atrous_rates=[6,12,18])
"""

class ASPPPooling(nn.Sequential):
    def __init__(self, in_channel, out_channel):
        super(ASPPPooling, self).__init__(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_channel, out_channel, 1, bias=False),
            nn.BatchNorm2d(out_channel),
            nn.ReLU())

    def forward(self, x):
        size = x.shape[-2:]
        for mod in self:
            x = mod(x)
        return F.interpolate(x, size=size, mode='bilinear', align_corners=False)


class ASPPConv(nn.Module):
    def __init__(self, in_channel, out_channel, atrous_rates=[6,12,18]):
        super(ASPPConv, self).__init__()
        layers = []
        layer_num = 2
        # 1x1 conv part
        layers.append(nn.Sequential(nn.Conv2d(in_channel, out_channel, 1, bias=False),
                                    nn.BatchNorm2d(out_channel),
                                    nn.ReLU()))
        # atrous conv part
        for atrous_rate in atrous_rates:
            layers.append(AtrousConv(in_channel,out_channel,atrous_rate))
            layer_num+=1
        # pooling part
        layers.append(ASPPPooling(in_channel, out_channel))
        self.convs = nn.ModuleList(layers)

        self.project = nn.Sequential(
            nn.Conv2d(layer_num * out_channel, out_channel, 1, bias=False),
            nn.BatchNorm2d(out_channel),
            nn.ReLU(),
            nn.Dropout(0.5))

    def forward(self, x):
        res = []
        for conv in self.convs:
            res.append(conv(x))
        res = torch.cat(res, dim=1)
        return self.project(res)


class XceptionBlock(nn.Module):
    """
    channel: no change
    size: 1/2
    """
    def __init__(self, in_channel,out_channels):
        super(XceptionBlock, self).__init__()
        self.shortcut=nn.Conv2d(in_channel,out_channels[-1],
                                kernel_size=1,
                                stride=2,
                                padding=0)
        layers = []
        layers.append(nn.ReLU())
        layers.append(SeparableConv2d(in_channel, out_channels[0],
                                    kernel_size=3,
                                    stride=1,
                                    padding=1,
                                    bias=False,
                                    act=0))
        for i in range(len(out_channels)-1):
            layers.append(SeparableConv2d(out_channels[i], out_channels[i+1],
                                        kernel_size=3,
                                        stride=1,
                                        padding=1,
                                        bias=False,
                                        act=0))
        self.sep_conv1 = nn.Sequential(*layers)

        self.sep_conv2 = SeparableConv2d(out_channels[-2], out_channels[-1],
                                    kernel_size=3,
                                    stride=2,
                                    padding=1,
                                    bias=False,
                                    act=0)

    def forward(self,x):
        shortcut = self.shortcut(x)
        sep_conv1 = self.sep_conv1(x)
        sep_conv2 = self.sep_conv2(sep_conv1)
        # print(shortcut.shape)
        # print(sep_conv.shape)
        return shortcut + sep_conv2, sep_conv1


class Decoder(nn.Module):
    def __init__(self,short_channel):
        super(Decoder, self).__init__()
        self.shortcut_conv = nn.Conv2d(256,short_channel
                            ,kernel_size=3
                            ,stride=1
                            ,padding=1
                            ,dilation=1
                            ,bias=False)
        layers = []
        layers.append(SeparableConv2d(cfg.ASPP_OUT_CHANNEL+short_channel, 256,
                                    kernel_size=3,
                                    stride=1,
                                    padding=1,
                                    bias=False,
                                    act=1))
        layers.append(SeparableConv2d(256, 256,
                                    kernel_size=3,
                                    stride=1,
                                    padding=1,
                                    bias=False,
                                    act=1))
        self.decoder_conv = nn.Sequential(*layers)

    def forward(self,encoder_feature,shortcut):
        shortcut = self.shortcut_conv(shortcut)
        encoder_feature = F.interpolate(encoder_feature,
                                        size=(shortcut.shape[-2],shortcut.shape[-1]),
                                        mode='bilinear',
                                        align_corners=False)
        encoder_feature = torch.cat((encoder_feature,shortcut),1)
        decoder_feature = self.decoder_conv(encoder_feature)
        return decoder_feature


class Deeplabv3plus(nn.Module):
    def __init__(self, in_channel=3, class_num=1000):
        super(Deeplabv3plus, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channel,
                            out_channels=32,
                            kernel_size=3,
                            stride=2,
                            padding=1,
                            bias=False)
        self.conv2 = nn.Conv2d(in_channels=32,
                            out_channels=64,
                            kernel_size=3,
                            stride=1,
                            padding=1,
                            bias=False)
        self.shortcut3 = XceptionBlock(64,[128,128,128])
        self.shortcut4 = XceptionBlock(128,[256,256,256])
        self.shortcut5 = XceptionBlock(256,[728,728,728])

        middle_layers = []
        for i in range(16):
            middle_layers.append(SeparableConv2d(728, 728,
                                    kernel_size=3,
                                    stride=1,
                                    padding=1,
                                    bias=False,
                                    act=0))
        self.middle = nn.Sequential(*middle_layers)

        self.shortcut7 = XceptionBlock(728,[728,1024,1024])
        self.sep_conv8 = SeparableConv2d(1024, 1536,
                                    kernel_size=3,
                                    stride=1,
                                    padding=1,
                                    bias=False,
                                    act=0)
        self.sep_conv9 = SeparableConv2d(1536, 1536,
                                    kernel_size=3,
                                    stride=1,
                                    padding=1,
                                    bias=False,
                                    act=0)
        self.sep_conv10 = SeparableConv2d(1536, 2048,
                                    kernel_size=3,
                                    stride=1,
                                    padding=1,
                                    bias=False,
                                    act=0)
        
        self.encoder = ASPPConv(2048,cfg.ASPP_OUT_CHANNEL)
        
        self.decoder = Decoder(short_channel=cfg.SHORTCUT_CHANNEL)
        self.predictor = nn.Sequential(nn.Conv2d(256, class_num,
                                                kernel_size=3,
                                                stride=1,
                                                padding=1,
                                                bias=False))

    def forward(self, x):
        
        out_conv1 = self.conv1(x)
        out_conv2 = self.conv2(out_conv1)
        out_shortcut3,_ = self.shortcut3(out_conv2)
        out_shortcut4, out_shortcut = self.shortcut4(out_shortcut3)
        out_shortcut5,_ = self.shortcut5(out_shortcut4)
        out_middle = self.middle(out_shortcut5)
        out_shortcut7,_ = self.shortcut7(out_middle)
        out_sep_conv8 = self.sep_conv8(out_shortcut7)
        out_sep_conv9 = self.sep_conv9(out_sep_conv8)
        out_sep_conv10 = self.sep_conv10(out_sep_conv9)

        encoder_feature = self.encoder(out_sep_conv10)
        decoder_feature = self.decoder(encoder_feature,out_shortcut)
        logit = self.predictor(decoder_feature)
        logit = F.interpolate(logit,
                            size=(x.shape[-2],x.shape[-1]),
                            mode='bilinear',
                            align_corners=False)
        return logit

# unet
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    layers = []
    layers.append(nn.Conv2d(in_planes, out_planes, 
                        kernel_size=3, padding=1, stride=stride))
    layers.append(nn.BatchNorm2d(out_planes))
    layers.append(nn.ReLU())
    return nn.Sequential(*layers)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution without padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride)

def up_conv2x2(in_planes,out_planes):

    return nn.ConvTranspose2d(in_planes,out_planes,kernel_size=2,stride=2)

def max_pool2x2():
    return nn.MaxPool2d(kernel_size=2,stride=2)

def match_size(src_tensor,dst_size):
    return F.interpolate(src_tensor, size=dst_size, mode='bilinear', align_corners=False)

class UNetv1(nn.Module):
    def __init__(self,in_channel=3, class_num=1000):
        super(UNetv1, self).__init__()
        # downsample stage
        self.conv_1 = nn.Sequential(conv3x3(in_channel,64),conv3x3(64,64))
        self.conv_2 = nn.Sequential(conv3x3(64,128),conv3x3(128,128))
        self.conv_3 = nn.Sequential(conv3x3(128,256),conv3x3(256,256))
        self.conv_4 = nn.Sequential(conv3x3(256,512),conv3x3(512,512))
        self.conv_5 = nn.Sequential(conv3x3(512,1024),conv3x3(1024,1024))
        self.maxpool = max_pool2x2()
        
        # upsample stage
        # up_conv_4 corresponds conv_4
        self.up_conv_4 = nn.Sequential(up_conv2x2(1024,512))
        # conv the cat(stage_4,up_conv_4) from 1024 to 512
        self.conv_6 = nn.Sequential(conv3x3(1024,512),conv3x3(512,512))
        # up_conv_3 corresponds conv_3
        self.up_conv_3 = nn.Sequential(up_conv2x2(512,256))
        # conv the cat(stage_3,up_conv_3) from 512 to 256
        self.conv_7 = nn.Sequential(conv3x3(512,256),conv3x3(256,256))
        # up_conv_2 corresponds conv_2
        self.up_conv_2 = nn.Sequential(up_conv2x2(256,128))
        # conv the cat(stage_2,up_conv_2) from 256 to 128
        self.conv_8 = nn.Sequential(conv3x3(256,128),conv3x3(128,128))
        # up_conv_1 corresponds conv_1
        self.up_conv_1 = nn.Sequential(up_conv2x2(128,64))
        # conv the cat(stage_1,up_conv_1) from 128 to 64
        self.conv_9 = nn.Sequential(conv3x3(128,64),conv3x3(64,64))
        # output
        self.result = conv1x1(64,class_num)

    def forward(self,x):
        # get 4 stage conv output
        stage_1 = self.conv_1(x)
        size_1 = stage_1.shape[-2:]
        # print(size_1)
        stage_2 = self.conv_2(self.maxpool(stage_1))
        size_2 = stage_2.shape[-2:]
        # print(size_2)
        stage_3 = self.conv_3(self.maxpool(stage_2))
        size_3 = stage_3.shape[-2:]
        # print(size_3)
        stage_4 = self.conv_4(self.maxpool(stage_3))
        size_4 = stage_4.shape[-2:]
        # print(size_4)

        # get up_conv_4 and concat with stage_4
        up_in_4 = self.conv_5(self.maxpool(stage_4))
        up_stage_4 = self.up_conv_4(up_in_4)
        up_stage_4 = match_size(up_stage_4,size_4)
        up_stage_4 = torch.cat((stage_4,up_stage_4),1)
        # get up_conv_3 and concat with stage_3
        up_in_3 = self.conv_6(up_stage_4)
        up_stage_3 = self.up_conv_3(up_in_3)
        up_stage_3 = match_size(up_stage_3,size_3)
        up_stage_3 = torch.cat((stage_3,up_stage_3),1)
        # get up_conv_2 and concat with stage_2
        up_in_2 = self.conv_7(up_stage_3)
        up_stage_2 = self.up_conv_2(up_in_2)
        up_stage_2 = match_size(up_stage_2,size_2)
        up_stage_2 = torch.cat((stage_2,up_stage_2),1)
        # get up_conv_1 and concat with stage_1
        up_in_1 = self.conv_8(up_stage_2)
        up_stage_1 = self.up_conv_1(up_in_1)
        up_stage_1 = match_size(up_stage_1,size_1)
        up_stage_1 = torch.cat((stage_1,up_stage_1),1)

        # last conv to channel 2
        out = self.conv_9(up_stage_1)
        # result
        out = self.result(out)
        return out

batch train val

In [0]:
def train_epoch(net, epoch, dataLoader, optimizer, criterion, train_log):
    # set to train 
    net.train()
    total_loss = 0.0
    dataprocess = tqdm(dataLoader)
    
    for batch_item in dataprocess:
        # get batch data
        batch_image, batch_label = batch_item['image'], batch_item['label']
        if torch.cuda.is_available():
            batch_image, batch_label = batch_image.cuda(), batch_label.cuda()
        # zero the gradient buffers
        optimizer.zero_grad()
        # forward to get output
        batch_out = net(batch_image)
        # compute loss
        batch_loss = criterion(batch_out, batch_label)
        total_loss += batch_loss.item()
        # auto backward
        batch_loss.backward()
        # update weights
        optimizer.step()
        # print batch result
        dataprocess.set_description_str("epoch:{}".format(epoch))
        dataprocess.set_postfix_str("batch_loss:{:.4f}".format(batch_loss.item()))

    # when opoch finished write to log file
    # log_title = 'epoch,average loss'
    log_string = "{},{:.4f} \n".format(epoch, total_loss / len(dataLoader))
    train_log.write(log_string)
    train_log.flush()


def eval_epoch(net, epoch, dataLoader, eval_log):
    # set to eval 
    net.eval()
    total_loss = 0.0
    dataprocess = tqdm(dataLoader)
    # init confusion matrix with all zeros
    confusion_matrix = {"TP": {i:0 for i in range(8)},
                        "TN": {i:0 for i in range(8)},
                        "FP": {i:0 for i in range(8)},
                        "FN": {i:0 for i in range(8)}}

    for batch_item in dataprocess:
        # get batch data
        batch_image, batch_label = batch_item['image'], batch_item['label']
        if torch.cuda.is_available():
            batch_image, batch_label = batch_image.cuda(), batch_label.cuda()
        # forward to get output
        batch_out = net(batch_image)
        # compute loss
        batch_loss = MySoftmaxCrossEntropyLoss(cfg.CLASS_NUM)(batch_out, batch_label)
        total_loss += batch_loss.detach().item()
        
        # get prediction,shape and values type same as batch_label

        # softmax: (1,0,0)->(0.89,0.06,0.05)
        # argmax: (0.89,0.06,0.05)->return index 0 
        pred = torch.argmax(F.softmax(batch_out, dim=1), dim=1)
        # compute confusion matrix
        confusion_matrix = update_confusion_matrix(pred, batch_label, confusion_matrix)
        # print batch result
        dataprocess.set_description_str("epoch:{}".format(epoch))
        dataprocess.set_postfix_str("batch_loss:{:.4f}".format(batch_loss))

    eval_loss = total_loss / len(dataLoader)
    
    # compute metric
    epoch_ious = compute_iou(confusion_matrix)
    epoch_m_iou = compute_mean(epoch_ious)
    epoch_precisions = compute_precision(confusion_matrix)
    epoch_m_precision = compute_mean(epoch_precisions)
    epoch_recalls = compute_recall(confusion_matrix)
    epoch_m_recall = compute_mean(epoch_recalls)
    
    # print eval iou every epoch
    print('mean iou: {} \n'.format(epoch_m_iou))
    for i in range(8):
        print_string = "iou_class_{} : {:.4f}\n".format(i, epoch_ious[i])
        print(print_string)
    
    # make log string
    log_values = [epoch,eval_loss,epoch_m_iou]+epoch_ious
    log_values += [epoch_m_precision]+epoch_precisions+[epoch_m_recall]+epoch_recalls
    log_values = [str(v) for v in log_values]
    log_string = ','.join(log_values)
    eval_log.write(log_string+'\n')
    eval_log.flush()


main train

In [0]:
# using multiprocesss to load data when cuda is available 
kwargs = {'num_workers': 4, 'pin_memory': True} if torch.cuda.is_available() else {}

# set image augment
augments = [ImageAug(),ToTensor()]

# get dataset and iterable dataloader
train_dataset = LaneSegTrainDataset("train.csv",transform=transforms.Compose(augments))
train_data_batch = DataLoader(train_dataset, batch_size=cfg.TRAIN_BATCH_SIZE, 
                              shuffle=True, drop_last=True, **kwargs)

eval_dataset = LaneSegTrainDataset("eval.csv",transform=transforms.Compose([ToTensor()]))
eval_data_batch = DataLoader(eval_dataset, batch_size=cfg.EVAL_BATCH_SIZE, 
                             shuffle=False, drop_last=False, **kwargs)

# define model

if cfg.MODEL == 'deeplabv3+':
    net = Deeplabv3plus(class_num=cfg.CLASS_NUM)
    
elif cfg.MODEL == 'unet':
    net = UNetv1(class_num=cfg.CLASS_NUM)
else:
    net = UNetv1(class_num=cfg.CLASS_NUM)
# load pretrained weights
if cfg.PRE_TRAINED:
    checkpoint = torch.load(os.path.join(cfg.LOG_DIR,cfg.PRE_TRAIN_WEIGHTS))
    net.load_state_dict(checkpoint['state_dict'])
# use cuda if available
if torch.cuda.is_available():
    net = net.cuda()

# define optimizer
# optimizer = torch.optim.SGD(net.parameters(), lr=cfg.BASE_LR, momentum=0.9, weight_decay=cfg.WEIGHT_DECAY)
optimizer = torch.optim.Adam(net.parameters(), lr=cfg.BASE_LR, weight_decay=cfg.WEIGHT_DECAY)
criterion = MySoftmaxCrossEntropyLoss(cfg.CLASS_NUM)

# define log file
train_log = open(os.path.join(cfg.LOG_DIR, "train_log_{0}_{1}th.csv".format(cfg.MODEL,cfg.TRAIN_NUMBER)), 'w')
train_log_title = "epoch,average loss\n"
train_log.write(train_log_title)
train_log.flush()
eval_log = open(os.path.join(cfg.LOG_DIR, "eval_log_{0}_{1}th.csv".format(cfg.MODEL,cfg.TRAIN_NUMBER)), 'w')
eval_log_title = "epoch,average_loss,\
                 mean_iou,\
                 iou_0,iou_1,iou_2,iou_3,iou_4,iou_5,iou_6,iou_7,\
                 mean_precision,\
                 precision_0,precision_1,precision_2,precision_3,precision_4,precision_5,precision_6,precision_7,\
                 mean_recall,\
                 recall_0,recall_1,recall_2,recall_3,recall_4,recall_5,recall_6,recall_7\n"
eval_log.write(eval_log_title)
eval_log.flush()
# train and test epoch by epoch
for epoch in range(cfg.EPOCHS):
    train_epoch(net, epoch, train_data_batch, optimizer, criterion, train_log)
    torch.save({'state_dict': net.state_dict()},
                   os.path.join(cfg.LOG_DIR,
                                "laneNet_{0}_{1}th_epoch_{2}.pth.tar".format(cfg.MODEL,cfg.TRAIN_NUMBER,epoch)))
    eval_epoch(net, epoch, eval_data_batch, eval_log)
        
    
train_log.close()
eval_log.close()
torch.save({'state_dict': net.state_dict()},
           os.path.join(cfg.LOG_DIR, 
                        "laneNet_{0}_{1}th.pth.tar".format(cfg.MODEL,cfg.TRAIN_NUMBER)))